<a href="https://colab.research.google.com/github/Samir-atra/CancerDetector/blob/main/Cancer_Detector/Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import PIL.Image
from PIL import ImageOps
import PIL
import pathlib
import matplotlib.pyplot as plt
import datetime
import tensorboard
import IPython
import sklearn
import cv2
import subprocess
import sys
import time

In [2]:
# Mounting google drive for training in google co lab

from google.colab import drive             
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Dataset loading

data_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/')

data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')
AUTOTUNE=tf.data.AUTOTUNE

start = time.time()
dataset_path = tf.keras.utils.image_dataset_from_directory(        # Training dataset
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='training',
    seed= 1,
    batch_size=5,
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

dataset_path_val = tf.keras.utils.image_dataset_from_directory(      #Validation dataset
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='validation',
    seed= 2,
    batch_size=5,
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

dataset_path_test = tf.keras.utils.image_dataset_from_directory(      # Testing dataset
    data_path_test,
    labels= 'inferred',
    seed= 3,
    batch_size=5,
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

end = time.time()
print("time elapsed = ", (end - start))

Found 8582 files belonging to 4 classes.
Using 6866 files for training.
Found 8582 files belonging to 4 classes.
Using 1716 files for validation.
Found 1705 files belonging to 4 classes.
time elapsed =  31.0553879737854


In [4]:
# catch and prefetch for shorter training time

# AUTOTUNE=tf.data.AUTOTUNE
# dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
# dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [5]:
# downloading the weights oft the base model

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (299, 299, 3),
    include_top = False, 
    weights = "imagenet"                  
)

87910968/87910968 [==============================] - 5s 0us/step


In [6]:
#saving the downloaded base_model 

saving_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedBaseModel.h5')

base_model.save(saving_path)

In [7]:
#Model

model_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedBaseModel.h5')

base_model = tf.keras.models.load_model(model_path)                 #Loading base_model

base_model.trainable = False                                        # Setting the model as non-trainable

rescaling = tf.keras.Sequential([                                        
  tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)             # Rescaling to (1, -1) range required for inceptionV3 model
])
augmentation = tf.keras.Sequential([                                # Applying augmentations o the images
   tf.keras.layers.RandomFlip("horizontal_and_vertical"),
   tf.keras.layers.RandomRotation(0.2)
])

inputs = tf.keras.Input(shape=(299, 299, 3))
# x = augmentation(inputs)
x = rescaling(inputs)
x = base_model(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)     # Neurons numbers and activation functions based on testing and choosing the one with best results
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(512, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)     # Regularizing and Dropout to avoid overfitting
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(512, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)                   # "softmax" in the final layer for decision making
model = tf.keras.Model(inputs, outputs)


model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),        # compiling with low learning rate
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)                                               

model.fit(                                                               # fitting the whole model for non-trainable base
    dataset_path,
    epochs=15,   
    validation_data = dataset_path_val)
    #verbose = 1)


model.evaluate(dataset_path_test, batch_size=5, verbose=1)               # evaluating using the test dataset 

model.summary()

base_model.trainable = True                                           # switching the base_model to trainable

for layer in base_model.layers:                                       # Swtching all batchnormalization layer of the base_model to non-trainable to not lose weights
  if isinstance(layer, tf.keras.layers.BatchNormalization):
    layer.trainable = False

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001),    # compiling the model with a trainable base_model
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)

model.fit(                                                            # fitting for another 10 epochs starting where the non-trainable ended
    dataset_path,
    epochs=30,         
    initial_epoch=15,                
    validation_data = dataset_path_val)
    #verbose = 1)

model.evaluate(dataset_path_test, batch_size=5, verbose=1)

model.summary()

saving_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedModelWithFT/')

tf.keras.models.save_model(model,                                     # saving the fully trained model
                           saving_path,
                           overwrite=True,
                           save_format='tf'
                           )

Epoch 1/15


1374/1374 [==============================] - 2122s 2s/step - loss: 3.3157 - accuracy: 0.7850 - val_loss: 2.9318 - val_accuracy: 0.8875
Epoch 2/15
1374/1374 [==============================] - 476s 346ms/step - loss: 2.8405 - accuracy: 0.8640 - val_loss: 2.5305 - val_accuracy: 0.9254
Epoch 3/15
1374/1374 [==============================] - 475s 346ms/step - loss: 2.4715 - accuracy: 0.8874 - val_loss: 2.2540 - val_accuracy: 0.9038
Epoch 4/15
1374/1374 [==============================] - 475s 345ms/step - loss: 2.1595 - accuracy: 0.8988 - val_loss: 1.8942 - val_accuracy: 0.9551
Epoch 5/15
1374/1374 [==============================] - 477s 347ms/step - loss: 1.8430 - accuracy: 0.9197 - val_loss: 1.7265 - val_accuracy: 0.9149
Epoch 6/15
1374/1374 [==============================] - 476s 347ms/step - loss: 1.5907 - accuracy: 0.9238 - val_loss: 1.4597 - val_accuracy: 0.9330
Epoch 7/15
1374/1374 [==============================] - 475s 346ms/step - loss: 1.3798 - accuracy: 0.9241 - val_loss: 1.2363 

May find a sample trained model at:  https://drive.google.com/drive/folders/1MsQzyfIj4JxdUIpP_EftdTRvWjm0YY8A?usp=sharing